<h1><center>Laboratorio 6: Optimización de modelos 🧪</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2025</strong></center>

### Cuerpo Docente:

- Profesores: Diego Cortez, Gabriel Iturra
- Auxiliares: Melanie Peña, Valentina Rojas
- Ayudantes: Nicolás Cabello, Cristopher Urbina

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Daniel Vargas
- Nombre de alumno 2: Rafael Tapia


Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: Entregas Martes a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria, pero se recomienda **fuertemente** asistir.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

### **Link de repositorio de GitHub:** [Insertar Repositorio](https://github.com/rodusflitope/DataScienceLabs/tree/lab6)

### Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.


### Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: 6 días de plazo con descuento de 1 punto por día. Entregas Martes a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria, pero se recomienda fuertemente asistir.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# Importamos librerias útiles

In [54]:
!pip install -qq xgboost optuna

# El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Al ver el gran potencial y talento que usted ha demostrado en el campo de la ciencia de datos, Fiu lo contrata como data scientist para que forme parte de su nuevo emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv('sales.csv')

df.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


## 1 Generando un Baseline (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad. [0.5 puntos]
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas. [1 punto]
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas. `Nota:` Utilice el método `.set_output(transform='pandas')` para obtener un DataFrame como salida del `ColumnTransformer` [1 punto]
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios. [0.5 punto]
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio? [0.5 puntos]
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`? [1 punto]
7. Guarde ambos modelos en un archivo .pkl (uno cada uno) [0.5 puntos]

In [2]:
from sklearn import set_config
set_config(transform_output="pandas")

from sklearn.model_selection import train_test_split

RANDOM_STATE = 10

train_df, test_df = train_test_split(df, test_size=0.1, random_state=RANDOM_STATE)
train_df, val_df = train_test_split(train_df, test_size=2/9, random_state=RANDOM_STATE)

target_column = 'quantity'

X_train = train_df.drop(target_column, axis=1)
y_train = train_df[target_column]

X_val = val_df.drop(target_column, axis=1)
y_val = val_df[target_column]

X_test = test_df.drop(target_column, axis=1)
y_test = test_df[target_column]

In [3]:
from sklearn.preprocessing import FunctionTransformer

def extract_date_features(X):
    X_copy = X.copy()
    # Convertir la columna 'date' a datetime si no lo está
    X_copy['date'] = pd.to_datetime(X_copy['date'], format='%d/%m/%y')
    # Extraer día, mes y año
    X_copy['day'] = X_copy['date'].dt.day.astype('category')
    X_copy['month'] = X_copy['date'].dt.month.astype('category')
    X_copy['year'] = X_copy['date'].dt.year.astype('category')
    # Eliminamos la fecha original para no OHE-encodearla completa porque da warning
    X_copy = X_copy.drop(columns=['date'])
    return X_copy

date_transformer = FunctionTransformer(
    extract_date_features
)

X_train_dates = date_transformer.fit_transform(X_train)

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

numeric_features = X_train_dates.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X_train_dates.select_dtypes(include=['category', 'object']).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),  # Escalar variables numéricas porque no dice que hacer con ellas
        ('cat', OneHotEncoder(sparse_output=False), categorical_features)
    ],
    remainder='passthrough'  # Mantener cualquier columna restante por si se me olvida algun tipo de col
)

preprocessor.set_output(transform="pandas")

,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,copy,True
,with_mean,True
,with_std,True


In [5]:
from sklearn.dummy import DummyRegressor
from sklearn.pipeline import Pipeline

baseline_pipeline = Pipeline([
    ('date_features', date_transformer), 
    ('preprocessor', preprocessor),
    ('regressor', DummyRegressor(strategy='mean'))
])

In [6]:
baseline_pipeline.fit(X_train, y_train)

,steps,"[('date_features', ...), ('preprocessor', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,func,<function ext...00199C254A700>
,inverse_func,None
,validate,False
,accept_sparse,False
,check_inverse,True
,feature_names_out,None
,kw_args,None


In [7]:
from sklearn.metrics import mean_absolute_error

y_val_pred = baseline_pipeline.predict(X_val)
mae_baseline = mean_absolute_error(y_val, y_val_pred)

print(f'MAE del modelo baseline en validacion: {mae_baseline:.2f}')

MAE del modelo baseline en validacion: 13176.15


Un MAE de 13176 significa que en promedio esto es lo que se equivoca mi modelo en relación al valor real, es decir, si predigo un valor k, entonces espero que me equivoque en 13176 unidades con respecto al valor real.

In [8]:
from xgboost import XGBRegressor
import joblib

xgb_pipeline = Pipeline([
    ('date_features', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(
        random_state=RANDOM_STATE,
    ))
])

xgb_pipeline.fit(X_train, y_train)

y_val_pred_xgb = xgb_pipeline.predict(X_val)
mae_xgb = mean_absolute_error(y_val, y_val_pred_xgb)

print(f"MAE XGBRegressor validacion: {mae_xgb:.2f}")

joblib.dump(baseline_pipeline, "baseline_dummy.pkl")
joblib.dump(xgb_pipeline, "xgb_default.pkl")

MAE XGBRegressor validacion: 2545.91


['xgb_default.pkl']

Bajo muchisimo el MAE a 2546, esto significa que el modelo en promedio se acerca mucho más al valor real pues epero equivocarme eso al hacer una predicción de la cantidad de compra de algo.

## 2. Forzando relaciones entre parámetros con XGBoost (10 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo realizando las siguientes tareas:

1. Vuelva a entrenar el `Pipeline` con `XGBRegressor`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Para aplicar esta restricción apóyese en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>. [6 puntos]

>Hint 1: Para implementar el constraint se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

>Hint 2: Puede obtener el nombre de las columnas en el paso anterior al modelo regresor mediante el método `.get_feature_names_out()`

2. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. [1 puntos]

3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo? [2 puntos]

4. Guarde su modelo en un archivo .pkl [1 punto]

In [9]:
xgb_monotone_pipeline = Pipeline([
    ('date_features', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(
        monotone_constraints={'num__price': -1}, 
        random_state=RANDOM_STATE,
    ))
])

xgb_monotone_pipeline.fit(X_train, y_train)
mae_xgb_mon = mean_absolute_error(y_val, xgb_monotone_pipeline.predict(X_val))

print(f"MAE XGB default:      {mae_xgb:.2f}")
print(f"MAE XGB monotónico:   {mae_xgb_mon:.2f}")

joblib.dump(xgb_monotone_pipeline, "xgb_monotone_price_neg.pkl")

MAE XGB default:      2545.91
MAE XGB monotónico:   2510.53


['xgb_monotone_price_neg.pkl']

Tenia razón pues mejoro ligeramente la calidad de la predicción (según MAE)

## 1.3 Optimización de Hiperparámetros con Optuna (20 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se pide que su optimización considere lo siguiente:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)

Para ello se pide los siguientes pasos:
1. Implemente una función `objective()` que permita minimizar el `MAE` en el conjunto de validación. Use el método `.set_user_attr()` para almacenar el mejor pipeline entrenado. [10 puntos]
2. Fije el tiempo de entrenamiento a 5 minutos. [1 punto]
3. Optimizar el modelo y reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
4. Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados? [5 puntos]
5. Guardar su modelo en un archivo .pkl [1 punto]

In [22]:
import optuna
from optuna.samplers import TPESampler
from sklearn.base import clone
optuna.logging.set_verbosity(optuna.logging.WARNING)
# Inserte su código acá

#Esto es para que al construir el preprocesador en cada trial se pruebe con el min_frequency escogido
def build_preprocessor(min_freq):
    prep = clone(preprocessor)
    prep.set_params(cat__min_frequency=min_freq)
    return prep

def objective(trial):
    # Hiperparámetros a optimizar (rangos del enunciado)
    learning_rate = trial.suggest_float("learning_rate", 0.001, 0.1, log=True)
    n_estimators = trial.suggest_int("n_estimators", 50, 1000)
    max_depth = trial.suggest_int("max_depth", 3, 10)
    max_leaves = trial.suggest_int("max_leaves", 0, 100)
    min_child_weight = trial.suggest_int("min_child_weight", 1, 5)
    reg_alpha = trial.suggest_float("reg_alpha", 0.0, 1.0)
    reg_lambda = trial.suggest_float("reg_lambda", 0.0, 1.0)
    ohe_min_freq = trial.suggest_float("ohe_min_frequency", 0.0, 1.0)

    # Preprocessor con min_frequency escogido
    prep = build_preprocessor(ohe_min_freq)

    # Modelo XGB
    xgb = XGBRegressor(
        random_state=RANDOM_STATE,
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        max_leaves=max_leaves,
        min_child_weight=min_child_weight,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        monotone_constraints={'num__price': -1}
    )

    pipe = Pipeline([
        ('date_features', date_transformer),
        ('preprocessor', prep), # Uso el que tiene el hiperparametro, no el que esta fijo
        ('regressor', xgb)
    ])

    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_val)
    mae = mean_absolute_error(y_val, y_pred)

    # Guardar pipeline y mae en los atributos del trial
    trial.set_user_attr("pipeline", pipe)
    trial.set_user_attr("mae", mae)

    return mae

study = optuna.create_study(direction="minimize", sampler=TPESampler(seed=RANDOM_STATE))
study.optimize(objective, timeout=300, show_progress_bar=True)

best_trial = study.best_trial
best_mae = best_trial.value
best_pipeline = best_trial.user_attrs["pipeline"]

print(f"Trials ejecutados: {len(study.trials)}")
print(f"Mejor MAE validación: {best_mae:.2f}")
print("Mejores hiperparámetros:")
for k,v in best_trial.params.items():
    print(f"  {k}: {v}")

joblib.dump(best_pipeline, "xgb_optuna.pkl")

Best trial: 93. Best value: 2008.55:  100%|██████████| 05:02/05:00


Trials ejecutados: 136
Mejor MAE validación: 2008.55
Mejores hiperparámetros:
  learning_rate: 0.03587694736422289
  n_estimators: 915
  max_depth: 9
  max_leaves: 100
  min_child_weight: 4
  reg_alpha: 0.8371112697125426
  reg_lambda: 0.04751105932555697
  ohe_min_frequency: 0.019020612698596513


['xgb_optuna.pkl']

Los resultados mejoraron 
pues lo que se hace es probar combinaciones de hiperparámetros e ir guardando la combinación que tenga la mejor métrica (en este caso menor MAE). Por esto, si en los valores propuestos estan los que usamos como baseline, el resultado necesariamente es igual o mejor porque en peor caso se usan lo que ya se usaron.

**learning_rate**: tamaño del paso de optimización se usa normalmente entre ese rango.

**n_estimators**: número de árboles boosting, tiene sentido el tamaño pues más de 1000 puede ser muy costoso.

**max_depth**: profundidad máxima, es para evitar overfitting en los árboles, quizas podría aumentar pero depende de la cantidad de parámetros y tenemos pocos.

**max_leaves**: límite de hojas, también depende de la cantidad de valores posibles y parámetros.

**min_child_weight**: mínimo peso para hacer un split, tiene sentido el árbol pues es laxo y como es un mínimo dificilmente nos dejara fuera un buen valor. Podría aumentarse el rango.

**reg_alpha**: regularización L1 induce sparsity en pesos de hojas castigando, el rango tiene sentido pues es la ponderación que tiene esta reg y podría ser mayor a 1 pero sería muy exagerado.

**reg_lambda**: regularización L2 reduce magnitud de pesos, estabiliza, por lo mismo que la L1 el rango tiene sentido.

**ohe_min_frequency**: filtra categorías raras agrupándolas reduciendo dimensionalidad, tiene sentido el valor porque como es relativo, estoy abarcando todos los casos, desde no tener mínimo que es 0 a que todo sea el mismo que es 1.

## 4. Optimización de Hiperparámetros con Optuna y Prunners (17 puntos)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento? [2 puntos]
- Redefinir la función `objective()` utilizando `optuna.integration.XGBoostPruningCallback` como método de **Prunning** [10 puntos]
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos [1 punto]
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
- Guardar su modelo en un archivo .pkl [1 punto]

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

In [76]:
!pip install optuna-integration[xgboost]

   ---------------------------------------- 0.0/99.1 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/99.1 kB ? eta -:--:--
   ------------------------------------- -- 92.2/99.1 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 99.1/99.1 kB 1.1 MB/s eta 0:00:00


Existen 2 formas de prunning, el pre-prunning y el post-prunning, le pre-prunning es lo que ya hicimos que es podar el árbol de forma preventiva poniendole límites como max-depth, max-leaves, etc.

El post prunning es una vez ya construido el árbol se evalua que puedo eliminar sin afectar mucho el rendimiento del árbol y las elimino, esto aunque muchas veces empeora el rendimiento del árbol en el conjunto de entrenamieno y validación, evita overfitting y debería después el árbol ser menos complejo y generalizar mejor.

En nuestro caso nuestro prunning lo que hace es hacer el pre-prunning de detener pero a la misma optimización de hiperparámetros por lo que el modelo debe generalizar mejor después aunque empeore un poco según validación.

In [20]:
# Inserte su código acá
from optuna.integration import XGBoostPruningCallback

optuna.logging.set_verbosity(optuna.logging.WARNING)

def build_preprocessor(min_freq):
    prep = clone(preprocessor)
    prep.set_params(cat__min_frequency=min_freq)
    return prep

def objective_pruning(trial):
    # Hiperparámetros
    learning_rate = trial.suggest_float("learning_rate", 0.001, 0.1, log=True)
    n_estimators = trial.suggest_int("n_estimators", 50, 1000)
    max_depth = trial.suggest_int("max_depth", 3, 10)
    max_leaves = trial.suggest_int("max_leaves", 0, 100)
    min_child_weight = trial.suggest_int("min_child_weight", 1, 5)
    reg_alpha = trial.suggest_float("reg_alpha", 0.0, 1.0)
    reg_lambda = trial.suggest_float("reg_lambda", 0.0, 1.0)
    ohe_min_freq = trial.suggest_float("ohe_min_frequency", 0.0, 1.0)

    prep = build_preprocessor(ohe_min_freq)


    featurizer = Pipeline([
        ('date_features', date_transformer),
        ('preprocessor',  prep)
    ])
    featurizer.fit(X_train, y_train)

    X_val_fe = featurizer.transform(X_val)

    pruning_cb = XGBoostPruningCallback(trial, "validation_0-mae")

    xgb = XGBRegressor(
        random_state=RANDOM_STATE,
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        max_leaves=max_leaves,
        min_child_weight=min_child_weight,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        monotone_constraints={'num__price': -1},
        eval_metric="mae",
        callbacks=[pruning_cb]
    )

    pipe = Pipeline([
        ('date_features', date_transformer),
        ('preprocessor', prep),
        ('regressor', xgb)
    ])

    # Entrena con eval_set para que Optuna reciba métricas por iteración
    pipe.fit(
        X_train,
        y_train,
        regressor__eval_set=[(X_val_fe, y_val)],
        )

    pred = pipe.predict(X_val)
    mae = mean_absolute_error(y_val, pred)

    trial.set_user_attr("pipeline", pipe)
    trial.set_user_attr("mae", mae)
    return mae

study_prune = optuna.create_study(direction="minimize", sampler=TPESampler(seed=RANDOM_STATE))
study_prune.optimize(objective_pruning, timeout=300, show_progress_bar=True)

best_trial_p = study_prune.best_trial
best_mae_p = best_trial_p.value
best_pipeline_p = best_trial_p.user_attrs["pipeline"]

print(f"Trials ejecutados: {len(study_prune.trials)}")
print(f"Mejor MAE validación: {best_mae_p:.2f}")
print("Mejores hiperparámetros:")
for k,v in best_trial_p.params.items():
    print(f"  {k}: {v}")

joblib.dump(best_pipeline_p, "xgb_optuna_pruning.pkl")

   0%|          | 00:00/05:00

[0]	validation_0-mae:12936.86798
[1]	validation_0-mae:12698.28403
[2]	validation_0-mae:12472.10525
[3]	validation_0-mae:12256.06570
[4]	validation_0-mae:12052.66956
[5]	validation_0-mae:11858.01188
[6]	validation_0-mae:11677.08460
[7]	validation_0-mae:11508.20789
[8]	validation_0-mae:11350.23560
[9]	validation_0-mae:11195.80685
[10]	validation_0-mae:11053.46771
[11]	validation_0-mae:10920.13034
[12]	validation_0-mae:10791.18215
[13]	validation_0-mae:10670.71907
[14]	validation_0-mae:10558.44026
[15]	validation_0-mae:10453.95557
[16]	validation_0-mae:10350.27999
[17]	validation_0-mae:10254.37531
[18]	validation_0-mae:10163.36666
[19]	validation_0-mae:10077.88336
[20]	validation_0-mae:9996.95586
[21]	validation_0-mae:9918.49066
[22]	validation_0-mae:9842.67945
[23]	validation_0-mae:9773.82985
[24]	validation_0-mae:9705.30802
[25]	validation_0-mae:9639.73675
[26]	validation_0-mae:9578.89902
[27]	validation_0-mae:9521.08228
[28]	validation_0-mae:9466.66860
[29]	validation_0-mae:9418.12813


Best trial: 0. Best value: 8298.11:    0%|          | 00:00/05:00

[0]	validation_0-mae:13161.00559
[1]	validation_0-mae:13145.86464
[2]	validation_0-mae:13130.84355
[3]	validation_0-mae:13115.84825
[4]	validation_0-mae:13100.88652
[5]	validation_0-mae:13086.02844
[6]	validation_0-mae:13070.47899
[7]	validation_0-mae:13054.96307
[8]	validation_0-mae:13039.50897
[9]	validation_0-mae:13024.10713
[10]	validation_0-mae:13008.80108
[11]	validation_0-mae:12993.56139
[12]	validation_0-mae:12978.31344
[13]	validation_0-mae:12963.17276
[14]	validation_0-mae:12948.05089
[15]	validation_0-mae:12932.93237
[16]	validation_0-mae:12918.01537
[17]	validation_0-mae:12903.04652
[18]	validation_0-mae:12888.28525
[19]	validation_0-mae:12873.36785
[20]	validation_0-mae:12858.69928
[21]	validation_0-mae:12843.94732
[22]	validation_0-mae:12829.36604
[23]	validation_0-mae:12814.78141
[24]	validation_0-mae:12800.25486
[25]	validation_0-mae:12785.80293
[26]	validation_0-mae:12771.34748
[27]	validation_0-mae:12757.03837
[28]	validation_0-mae:12742.65899
[29]	validation_0-mae:12

Best trial: 0. Best value: 8298.11:    1%|          | 00:01/05:00

...

Best trial: 82. Best value: 2069.89:   98%|█████████▊| 04:58/05:00

[0]	validation_0-mae:12522.46411
[1]	validation_0-mae:11949.80012
[2]	validation_0-mae:11436.15147
[3]	validation_0-mae:10953.34929
[4]	validation_0-mae:10532.45201
[5]	validation_0-mae:10124.75498
[6]	validation_0-mae:9750.87253
[7]	validation_0-mae:9389.18062
[8]	validation_0-mae:9079.33506
[9]	validation_0-mae:8774.06709
[10]	validation_0-mae:8524.14417
[11]	validation_0-mae:8186.89935
[12]	validation_0-mae:7858.94600
[13]	validation_0-mae:7636.02443
[14]	validation_0-mae:7359.76735
[15]	validation_0-mae:7157.72442
[16]	validation_0-mae:6911.03258
[17]	validation_0-mae:6691.62824
[18]	validation_0-mae:6471.07393
[19]	validation_0-mae:6334.96288
[20]	validation_0-mae:6181.21112
[21]	validation_0-mae:5967.63592
[22]	validation_0-mae:5827.98313
[23]	validation_0-mae:5650.37593
[24]	validation_0-mae:5503.60678
[25]	validation_0-mae:5359.87538
[26]	validation_0-mae:5258.41811
[27]	validation_0-mae:5146.40260
[28]	validation_0-mae:5019.89633
[29]	validation_0-mae:4920.19590
[30]	validatio

Best trial: 82. Best value: 2069.89:  100%|██████████| 05:05/05:00


Trials ejecutados: 94
Mejor MAE validación: 2069.89
Mejores hiperparámetros:
  learning_rate: 0.085617042833164
  n_estimators: 599
  max_depth: 7
  max_leaves: 85
  min_child_weight: 1
  reg_alpha: 0.5031204644318659
  reg_lambda: 0.8071465184114838
  ohe_min_frequency: 0.06263425902727707


['xgb_optuna_pruning.pkl']

Podemos ver que el mejor MAE empeoro ligeramente de 2008 a 2069, siendo que podríamos esperar que como se probaron más combinaciones debería ser mejor, esto puede deberse a que algunos se cortaron antes de llegar a su mejor valor.

## 5. Visualizaciones (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

1. Gráfico de historial de optimización [1 punto]
2. Gráfico de coordenadas paralelas [1 punto]
3. Gráfico de importancia de hiperparámetros [1 punto]

Comente sus resultados:

4. ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? [0.5 puntos]
5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? [1 punto]
6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo? [0.5 puntos]

In [21]:
# Inserte su código acá
from optuna.visualization import (
    plot_optimization_history,
    plot_parallel_coordinate,
    plot_param_importances,
)

plot_optimization_history(study_prune).show()

plot_parallel_coordinate(study_prune).show()

plot_param_importances(study_prune).show()

4. Los primeros trials valen mucho más que los posteriores, más que desde que trial se empieza a notar mejoras notables, podemos ver que desde más o menos el trial número 10, las mejoras disminuyen muchísimo
5. en general los trials con valores bajos en mae tienen un learning rate cercano a 0.1, max_depth menor a 7, max_leaves mayor a 60, n_estimators mayor a 500, ohe_min_freq menor a 0.2, reg_alpha entre 0.2 y 0.6, y finalmente reg_lambda entre 0.6 y 0.8
6. por mucho son n_estimators y ohe_min_frequency

## 6. Síntesis de resultados (3 puntos)

Finalmente:

1. Genere una tabla resumen del MAE en el conjunto de validación obtenido en los 5 modelos entrenados desde Baseline hasta XGBoost con Constraints, Optuna y Prunning. [1 punto]
2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento? [0.5 puntos]
3. Cargue el mejor modelo, prediga sobre el conjunto de **test** y reporte su MAE. [0.5 puntos]
4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto? [1 punto]

In [24]:
# Inserte su código acá
modelos = {
    "Baseline": joblib.load("baseline_dummy.pkl"),
    "XGBoost": joblib.load("xgb_default.pkl"),
    "XGBoost + Monotone": joblib.load("xgb_monotone_price_neg.pkl"),
    "XGBoost + Optuna": joblib.load("xgb_optuna.pkl"),
    "XGBoost + Optuna + Pruning": joblib.load("xgb_optuna_pruning.pkl"),
}

resultados = []
for nombre, modelo in modelos.items():
    y_pred = modelo.predict(X_val)
    mae = mean_absolute_error(y_val, y_pred)
    resultados.append({"Modelo": nombre, "MAE_Validación": mae})

tabla_mae = pd.DataFrame(resultados)
display(tabla_mae)

,Modelo,MAE_Validación
0,Baseline,13176.150888
1,XGBoost,2545.905762
2,XGBoost + Monotone,2510.528076
3,XGBoost + Optuna,2008.553955
4,XGBoost + Optuna + Pruning,2069.894287


El mejor modelo es el de optuna sin pruning a nivel de tener el menor MAE

In [25]:
best_model = joblib.load("xgb_optuna.pkl")
y_pred_test = best_model.predict(X_test)
mae_test = mean_absolute_error(y_test, y_pred_test)
print(f"MAE en Test: {mae_test:.4f}")

MAE en Test: 2330.4026


Podemos ver que el MAE en Test es considerablemente pero que el de validación siendo de 2330 contra los 2008 de validación, esto ocurre porqe al momento de hacer una optimización de hiperparámetros estamos introduciendo un sesgo hacia los datos de validación de la misma forma que el modelo tiene un sesgo y se sobreajusta a los datos de entrenamiento, al momento de optimizar respecto a validación estamos sobreajustando hacia esos datos, por eso es tan importante probar solamente al final en los de test pues si intentamos ahora hacer algo para mejorar los resultados de test estamos introduciendo un sesgo hacia esos datos con respecto al universo real.

# Conclusión
Exito!
<p align="center">
  <img src="https://i.pinimg.com/originals/55/3d/42/553d42bea9b10e0662a05aa8726fc7f4.gif">
</p>